In [1]:
import os
from datetime import datetime
import tensorflow as tf
import numpy as np
from scipy.misc import imread, imresize
from imagenet_classes import class_names
from datagenerator import ImageDataGenerator

In [2]:
# Path to the textfiles for the trainings and validation set
train_file = '../../models/train_partition_finetuning_standard_AVA_balanced.txt'
test_file = '../../models/test_partition_finetuning_standard_AVA_balanced.txt'
train_generator = ImageDataGenerator(train_file, shuffle = False, scale_size=(224, 224))
test_generator = ImageDataGenerator(test_file, shuffle = False, scale_size=(224, 224)) 

# Learning params
lr = 0.001
num_epochs = 20
batch_size = 128

# Network params
fc_nodes = 1000
num_classes = 2
dropout_rate = 0.5
display_step = 10
checkpoint_path = '../../models/AesNet_VGG16_fc{}'.format(fc_nodes)

# Get the number of training/validation steps per epoch
train_batches_per_epoch = np.floor(train_generator.data_size / batch_size).astype(np.int16)
test_batches_per_epoch = np.floor(test_generator.data_size / batch_size).astype(np.int16)

In [3]:
parameters = []
g = tf.Graph()

with g.as_default():
    
    imgs = tf.placeholder(tf.float32, [None, 224, 224, 3])
    y = tf.placeholder(tf.float32, shape=[None, num_classes])
    
    keep_prob = tf.placeholder(tf.float32)
    learning_rate = tf.placeholder(tf.float32)
    
    # zero-mean input
    #with tf.name_scope('preprocess') as scope:
    #    mean = tf.constant([123.68, 116.779, 103.939], dtype=tf.float32, shape=[1, 1, 1, 3], name='img_mean')
    #    images = imgs-mean

    # conv1_1
    with tf.name_scope('conv1_1') as scope:
        weights = tf.Variable(tf.truncated_normal([3, 3, 3, 64], dtype=tf.float32,stddev=1e-1),trainable=False, name='weights')
        biases = tf.Variable(tf.constant(0.0, shape=[64], dtype=tf.float32),trainable=False, name='biases')
        
        conv1_1 = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(imgs, weights, [1, 1, 1, 1], padding='SAME'),biases), name=scope)
        
        parameters += [weights, biases]
        
    # conv1_2
    with tf.name_scope('conv1_2') as scope:
        weights = tf.Variable(tf.truncated_normal([3, 3, 64, 64], dtype=tf.float32, stddev=1e-1),trainable=False, name='weights')
        biases = tf.Variable(tf.constant(0.0, shape=[64], dtype=tf.float32), trainable=False, name='biases')
        
        conv1_2 = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(conv1_1, weights, [1, 1, 1, 1], padding='SAME'),biases), name=scope)
        
        parameters += [weights, biases]

    # pool1
    pool1 = tf.nn.max_pool(conv1_2, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1],padding='SAME',name='pool1')

    # conv2_1
    with tf.name_scope('conv2_1') as scope:
        weights = tf.Variable(tf.truncated_normal([3, 3, 64, 128], dtype=tf.float32,stddev=1e-1),trainable=False, name='weights')
        biases = tf.Variable(tf.constant(0.0, shape=[128], dtype=tf.float32),trainable=False, name='biases')
        
        conv2_1 = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(pool1, weights, [1, 1, 1, 1], padding='SAME'),biases), name=scope)
        
        parameters += [weights, biases]
        
    # conv2_2
    with tf.name_scope('conv2_2') as scope:
        weights = tf.Variable(tf.truncated_normal([3, 3, 128, 128], dtype=tf.float32,stddev=1e-1),trainable=False, name='weights')
        biases = tf.Variable(tf.constant(0.0, shape=[128], dtype=tf.float32),trainable=False, name='biases')
        
        conv2_2 = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(conv2_1, weights, [1, 1, 1, 1], padding='SAME'),biases), name=scope)
        
        parameters += [weights, biases]

    # pool2
    pool2 = tf.nn.max_pool(conv2_2,ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1],padding='SAME',name='pool2')

    # conv3_1
    with tf.name_scope('conv3_1') as scope:
        weights = tf.Variable(tf.truncated_normal([3, 3, 128, 256], dtype=tf.float32,stddev=1e-1),trainable=False, name='weights')
        biases = tf.Variable(tf.constant(0.0, shape=[256], dtype=tf.float32),trainable=False, name='biases')
        
        conv3_1 = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(pool2, weights, [1, 1, 1, 1], padding='SAME'),biases), name=scope)
        
        parameters += [weights, biases]

    # conv3_2
    with tf.name_scope('conv3_2') as scope:
        weights = tf.Variable(tf.truncated_normal([3, 3, 256, 256], dtype=tf.float32,stddev=1e-1),trainable=False, name='weights')
        biases = tf.Variable(tf.constant(0.0, shape=[256], dtype=tf.float32),trainable=False, name='biases')
        
        conv3_2 = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(conv3_1, weights, [1, 1, 1, 1], padding='SAME'),biases), name=scope)
        
        parameters += [weights, biases]

    # conv3_3
    with tf.name_scope('conv3_3') as scope:
        weights = tf.Variable(tf.truncated_normal([3, 3, 256, 256], dtype=tf.float32,stddev=1e-1),trainable=False, name='weights')
        biases = tf.Variable(tf.constant(0.0, shape=[256], dtype=tf.float32),trainable=False, name='biases')
        
        conv3_3 = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(conv3_2, weights, [1, 1, 1, 1], padding='SAME'),biases), name=scope)
        
        parameters += [weights, biases]

    # pool3
    pool3 = tf.nn.max_pool(conv3_3,ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1],padding='SAME',name='pool3')

    # conv4_1
    with tf.name_scope('conv4_1') as scope:
        weights = tf.Variable(tf.truncated_normal([3, 3, 256, 512], dtype=tf.float32,stddev=1e-1),trainable=False, name='weights')
        biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),trainable=False, name='biases')
        
        conv4_1 = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(pool3, weights, [1, 1, 1, 1], padding='SAME'),biases), name=scope)
        
        parameters += [weights, biases]

    # conv4_2
    with tf.name_scope('conv4_2') as scope:
        weights = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,stddev=1e-1),trainable=False, name='weights')
        biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),trainable=False, name='biases')
        
        conv4_2 = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(conv4_1, weights, [1, 1, 1, 1], padding='SAME'),biases), name=scope)
        
        parameters += [weights, biases]

    # conv4_3
    with tf.name_scope('conv4_3') as scope:
        weights = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,stddev=1e-1),trainable=False, name='weights')
        biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),trainable=False, name='biases')
        
        conv4_3 = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(conv4_2, weights, [1, 1, 1, 1], padding='SAME'),biases), name=scope)
        
        parameters += [weights, biases]

    # pool4
    pool4 = tf.nn.max_pool(conv4_3,ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1],padding='SAME',name='pool4')

    # conv5_1
    with tf.name_scope('conv5_1') as scope:
        weights = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,stddev=1e-1),trainable=False, name='weights')
        biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),trainable=False, name='biases')
        
        conv5_1 = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(pool4, weights, [1, 1, 1, 1], padding='SAME'),biases), name=scope)
        
        parameters += [weights, biases]

    # conv5_2
    with tf.name_scope('conv5_2') as scope:
        weights = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,stddev=1e-1),trainable=False, name='weights')
        biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),trainable=False, name='biases')
        
        conv5_2 = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(conv5_1, weights, [1, 1, 1, 1], padding='SAME'),biases), name=scope)
        
        parameters += [weights, biases]

    # conv5_3
    with tf.name_scope('conv5_3') as scope:
        weights = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,stddev=1e-1),trainable=False, name='weights')
        biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),trainable=False, name='biases')
        
        conv5_3 = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(conv5_2, weights, [1, 1, 1, 1], padding='SAME'),biases), name=scope)
        
        parameters += [weights, biases]

    # pool5
    pool5 = tf.nn.max_pool(conv5_3,ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1],padding='SAME',name='pool5')

    # fc6
    with tf.name_scope('fc6') as scope:
        shape = int(np.prod(pool5.get_shape()[1:]))
        weights = tf.Variable(tf.truncated_normal([shape, fc_nodes],dtype=tf.float32,stddev=1e-1), name='weights')
        biases = tf.Variable(tf.constant(1.0, shape=[fc_nodes], dtype=tf.float32),trainable=True, name='biases')
        
        pool5_flat = tf.reshape(pool5, [-1, shape])
        fc6 = tf.nn.relu(tf.nn.bias_add(tf.matmul(pool5_flat, weights), biases))
        
        parameters += [weights, biases]
        
        fc6_drop = tf.nn.dropout(fc6, keep_prob)

    # fc7
    with tf.name_scope('fc7') as scope:
        weights = tf.Variable(tf.truncated_normal([fc_nodes, fc_nodes],dtype=tf.float32, stddev=1e-1), name='weights')
        biases = tf.Variable(tf.constant(1.0, shape=[fc_nodes], dtype=tf.float32),trainable=True, name='biases')
        
        fc7 = tf.nn.relu(tf.nn.bias_add(tf.matmul(fc6_drop, weights), biases))
        
        parameters += [weights, biases]
        
        fc7_drop = tf.nn.dropout(fc7, keep_prob)

    # fc8 (output)
    with tf.name_scope('fc8') as scope:
        weights = tf.Variable(tf.truncated_normal([fc_nodes, num_classes],dtype=tf.float32,stddev=1e-1), name='weights')
        biases = tf.Variable(tf.constant(1.0, shape=[num_classes], dtype=tf.float32),trainable=True, name='biases')
        
        fc8 = tf.nn.bias_add(tf.matmul(fc7_drop, weights), biases)
        
        parameters += [weights, biases]
        
    # Op for calculating the loss
    with tf.name_scope("cross_ent"):
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = fc8, labels = y))  

    # Train op
    with tf.name_scope("train"):
        # Get gradients of all trainable variables
        var_list = tf.trainable_variables()
        #gradients = tf.gradients(loss, var_list)
        #gradients = list(zip(gradients, var_list))

        # Create optimizer and apply gradient descent to the trainable variables
        optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        gradients = optimizer.compute_gradients(loss, var_list)
        train_op = optimizer.apply_gradients(gradients)
        
    correct_prediction = tf.equal(tf.argmax(fc8, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    saver = tf.train.Saver()

In [ ]:
with tf.Session(graph=g) as sess:

    sess.run(tf.global_variables_initializer())
    
    weight_file ="vgg16_weights.npz"
    weights = np.load(weight_file)
    keys = sorted(weights.keys())
    for i, k in enumerate(keys):
        #print (i, parameters[i].name, k, np.shape(weights[k]))
        if (i<=25):
            sess.run(parameters[i].assign(weights[k]))
            
    for epoch in range(num_epochs):
        if(epoch > 0 and epoch%7==0):
            lr *= 0.1
            
        step = 1
        while step < train_batches_per_epoch:

            # Get a batch of images and labels
            batch_xs, batch_ys = train_generator.next_batch(batch_size)

            # And run the training op
            sess.run(train_op, feed_dict={imgs: batch_xs, 
                                          y: batch_ys, 
                                          keep_prob: dropout_rate,
                                          learning_rate: lr})
            step += 1
        
            if(step % 10 == 0):
                aux_accu = accuracy.eval(feed_dict={imgs: batch_xs, 
                                                        y: batch_ys, 
                                                        keep_prob: 1.,
                                                        learning_rate: lr})
                print(epoch+1, aux_accu)

                print("{} Saving checkpoint of model...".format(datetime.now()))  

                #save checkpoint of the model
                checkpoint_name = os.path.join(checkpoint_path, 'model_epoch'+str(epoch+1)+'.ckpt')
                save_path = saver.save(sess, checkpoint_name)  

                print("{} Model checkpoint saved at {}".format(datetime.now(), checkpoint_name))
            

In [4]:

with tf.Session(graph=g) as sess:
    # Restore variables from disk.
    saver.restore(sess, "../../models/AesNet_VGG16_fc1000/model_epoch20.ckpt")

    test_acc = 0.
    test_count = 0
    for _ in range(test_batches_per_epoch):
        batch_tx, batch_ty = test_generator.next_batch(batch_size)
        acc = sess.run(accuracy, feed_dict={imgs: batch_tx, y: batch_ty, keep_prob: 1.,learning_rate: lr})
        test_acc += acc
        test_count += 1
        print(test_count)
    test_acc /= test_count
    print("{} Test Accuracy = {:.4f}".format(datetime.now(), test_acc))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
2017-12-04 09:53:08.688691 Test Accuracy = 0.2887
